Прежде чем проверять задания:
- **Перезапустите ядро** (**restart the kernel**) (В меню, выбрать Ядро (Kernel) $\rightarrow$ Перезапустить (Restart)
- Затем **Выполнить все ячейки**  **run all cells** (В меню, выбрать Ячейка (Cell) $\rightarrow$ Запустить все (Run All).

Убедитесь, что заполнены все ячейки с комментарием "НАЧАЛО ВАШЕГО РЕШЕНИЯ".

После ячеек с заданием следуют ячейки с проверкой с помощью assert.

Если в коде есть ошибки, assert выведет уведомление об ошибке.

Если в коде нет ошибок, assert отработает без вывода дополнительной информации.

---

Датасет **"Недвижимость в Калифорнии"** (California Housing Prices dataset) содержит информацию о жилых районах в штате Калифорния, основанную на переписи населения 1990 года. Он часто используется в задачах регрессии для предсказания медианной стоимости жилья в районе на основе данных о населении, доходах, количестве комнат, географическом положении и других характеристиках района.

Датасет содержит 20 640 наблюдений и 9 признаков. Каждое наблюдение представляет собой информацию о жилом районе, а каждый признак описывает различные характеристики района:

- longitude: долгота географического положения района.
- latitude: широта географического положения района.
- housing_median_age: медианный возраст домов в районе.
- total_rooms: среднее количество комнат в доме в районе.
- total_bedrooms: среднее количество спален в доме в районе.
- population: общее количество людей, проживающих в районе.
- households: среднее количество людей, проживающих в каждом доме в районе.
- median_income: медианный доход домохозяйства в районе.
- median_house_value: медианная стоимость домов в районе (целевая переменная).
- ocean_proximity: близость к океану (категориальная переменная).

Целевая переменная MedHouseVal является непрерывной переменной, что делает этот датасет идеальным для задачи регрессии. Данные были собраны в 1990 году, поэтому медианные стоимости домов устарели и не отражают текущие рыночные цены на недвижимость.

Этот датасет доступен в библиотеке scikit-learn и также может быть загружен из репозитория UCI Machine Learning Repository. В нашем случае он загружается из файла.

In [2]:
# импорт библиотек
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

np.random.seed(42)  # фиксация генератора случайных чисел

In [3]:
# загрузка набора данных по ценам на недвижимость в Калифорнии
# https://drive.google.com/file/d/1rcfG4gq3EkS0mEng2l0MJHHFkMFsPoLV/view?usp=sharing
# ссылка на датасет, размещенные на гугл-диске
url = (
    "https://drive.google.com/file/d/1rcfG4gq3EkS0mEng2l0MJHHFkMFsPoLV/view?usp=sharing"
)
url = "https://drive.google.com/uc?id=" + url.split("/")[-2]
california_housing = pd.read_csv(url, sep=",")
california_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
"""
Задание 1. Мы видим, что в столбце "total_bedrooms" отсутствует 207 значений.
Давайте удалим эти 207 строк с помощью применения  метода .dropna с параметром inplace=True
к нашему датафрейму (california_housing).

Примените dropna к california_housing с параметром inplace=True
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
california_housing.dropna(axis=0, inplace=True)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка задания 1. Просто запустите ячейку.
assert california_housing.shape == (
    20433,
    10,
), "размер датасета не совпадает с правильным"
print(california_housing.shape)

(20433, 10)


In [9]:
"""
Задание 2. Разделим датафрейм california_housing на признаки и целевую переменную.
Метод drop() в библиотеке Pandas используется для удаления одного или нескольких
столбцов или строк из DataFrame. Метод drop() не изменяет исходный DataFrame,
а возвращает новый DataFrame с удаленными столбцами или строками.
Примените метод drop к california_housing, передав туда
в параметр columns значение ['median_house_value']. Получившийся DataFrame сохранить в
переменную с признаками california_housing_data
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
california_housing_data = california_housing.drop(columns=["median_house_value"])
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [10]:
# Проверка задания 2. Просто запустите ячейку.
assert california_housing_data.shape == (20433, 9)

In [11]:
"""
Задание 3. Операция df['column'] в pandas называется индексацией (indexing) и используется
для извлечения одного или нескольких столбцов из DataFrame.
Эта операция позволяет получить Series, соответствующий одному столбцу DataFrame.
Если вместо одного столбца нужно выбрать несколько столбцов, то следует передать
список с именами столбцов в квадратных скобках, например: df[['column1', 'column2']].

Примените операцию индексация к california_housing, передав туда
в качестве column значение 'median_house_value'. Получившийся Series сохранить в
переменную california_housing_target
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
california_housing_target = california_housing["median_house_value"]
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [12]:
# Проверка задания 3. Просто запустите ячейку.
assert california_housing_target.shape == (20433,)

In [13]:
"""
Задание 4. Посмотрите, сколько уникальных значений категорий содержится в колонке 'ocean_proximity' датафрейма
california_housing_data. Это можно сделать с помощью встроенной функции len и метода unique().
Ответ сохраните в переменную num_values (Пример num_values = len(df['column'].unique()))
"""
num_values = None

# НАЧАЛО ВАШЕГО РЕШЕНИЯ
num_values = california_housing_data["ocean_proximity"].nunique()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [14]:
# Проверка задания 4. Просто запустите ячейку.
assert num_values

In [18]:
"""
Задание 5. Преобразуйте категориальный признак 'ocean_proximity' в несколько бинарных признаков с помощью
One-Hot Encoding. Результат работы pd.get_dummies сохраните в ту же переменную california_housing_data
Пример: df = pd.get_dummies(df, columns=['color'])
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
california_housing_data = pd.get_dummies(
    california_housing_data, columns=["ocean_proximity"]
)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [19]:
# Проверка задания 5. Просто запустите ячейку.
print(california_housing_data.info())
# Мы видим, что исчез столбец ocean_proximity и появились новые столбцы

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitude                   20433 non-null  float64
 1   latitude                    20433 non-null  float64
 2   housing_median_age          20433 non-null  float64
 3   total_rooms                 20433 non-null  float64
 4   total_bedrooms              20433 non-null  float64
 5   population                  20433 non-null  float64
 6   households                  20433 non-null  float64
 7   median_income               20433 non-null  float64
 8   ocean_proximity_<1H OCEAN   20433 non-null  bool   
 9   ocean_proximity_INLAND      20433 non-null  bool   
 10  ocean_proximity_ISLAND      20433 non-null  bool   
 11  ocean_proximity_NEAR BAY    20433 non-null  bool   
 12  ocean_proximity_NEAR OCEAN  20433 non-null  bool   
dtypes: bool(5), float64(8)
memory usage:

In [20]:
"""
Задание 6. Разделите DataFrame california_housing_data и Series california_housing_target
на обучающую и тестовую выборки с помощью функции train_test_split.
Можно не указывать параметр test_size (тогда подставится значение по умолчанию 0.25),
либо указать напрямую test_size=0.25
Передайте в эту функцию признаки и целевую переменную.
Пример: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
"""
X_train, X_test, y_train, y_test = train_test_split(
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    california_housing_data,
    california_housing_target,
    test_size=0.25
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ
)

In [21]:
# Проверка задания 6. Просто запустите ячейку.
assert X_train.shape == (15324, 13)
assert y_test.shape == (5109,)

Мы будем искать оптимальный параметр 'n_neighbors' (количество соседей) с помощью GridSearchCV.
GridSearchCV - это инструмент для поиска наилучших гиперпараметров модели машинного обучения.
Он используется для автоматического подбора наилучших параметров модели путем перебора всех
возможных комбинаций гиперпараметров из заранее определенного диапазона.

Основная идея GridSearchCV заключается в том, чтобы создать сетку (grid) всех возможных комбинаций
гиперпараметров и обучить модель на каждой из этих комбинаций.
Затем GridSearchCV выбирает ту комбинацию гиперпараметров, которая показала наилучшую
производительность (например, наилучшее значение метрики качества на тестовых данных).

Этот инструмент особенно полезен при работе с моделями, у которых множество гиперпараметров,
и требуется максимально точно настроить их значения для достижения наилучшей производительности.

In [27]:
"""
Задание 7. Создайте словарь param_grid с одним ключом 'n_neighbors' и значением этого ключа,
равным [1, 3, 5, 7, 9]
Пример: dict_example = {'key': [1, 3, 5]}
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
param_grid = dict([("n_neighbors", [1, 3, 5, 7, 9])])
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [28]:
# Проверка задания 7. Просто запустите ячейку.
assert "n_neighbors" in param_grid

In [29]:
# создание экземпляра класса KNeighborsRegressor
knn = KNeighborsRegressor()

In [30]:
"""
Задание 8. Создайте экземпляр класса GridSearchCV для поиска оптимального значения 'n_neighbors'.
Передайте в него три параметра:
1. Экземпляр класса KNeighborsRegressor, который создан выше
2. Словарь param_grid
3. Параметр сv со значением 5 (Этот параметр отвечает за разбивку обучающей выборки на 5 частей
    в рамках кросс-валидации)

Пример: grid_search = GridSearchCV(regressor, parameters_grid, cv=10)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
grid_search = GridSearchCV(knn, param_grid, cv=5)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [31]:
# Проверка задания 8. Просто запустите ячейку.
grid_search_vars = vars(grid_search)
assert grid_search_vars["cv"] == 5

In [35]:
"""
Задание 9. Запустите обучение grid_search, передав в метод grid_search.fit()
ОБУЧАЮЩИЕ данные и целевые переменные
Пример: regressor.fit(X, y)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
grid_search.fit(X_train, y_train)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9]})

In [36]:
# Проверка задания 9. Просто запустите ячейку.
assert hasattr(grid_search, "best_params_")
# вывод оптимального значения параметра K
print("Оптимальное значение K:", grid_search.best_params_["n_neighbors"])

Оптимальное значение K: 9


In [37]:
# Прогнозирование на тестовой выборке осуществляется с использованием оптимального значения K.
y_pred = grid_search.predict(X_test)

In [38]:
"""
Задание 10. Осуществите вычисление mean_absolute_error и mean_squared_error на ТЕСТОВЫХ данных
Ответ запишите в переменные mae и mse
Пример:
mae = mean_absolute_error(y, pred)
mse = mean_squared_error(y, pred)
"""
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [39]:
# Проверка задания 10. Просто запустите ячейку.
assert int(mae) == 75819, "величина mae не совпадает с величиной из теста"
print("MAE:", mae)
print("MSE:", mse)

MAE: 75819.99536765185
MSE: 9433490759.921362
